<a href="https://colab.research.google.com/github/mvanitha/DE-M7/blob/main/Advanced_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Path chosen - Option A: Multi-Agent Systems**

In [2]:
# First, let's set up our imports and API key
from google.colab import userdata
import google.genai as genai
from google.genai import types
from IPython.display import Markdown, display, HTML

# Configure the API with your key
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

print("✓ Setup complete! Choose your path below.")

✓ Setup complete! Choose your path below.


In [3]:
# Agent 1: SQL Generator
generator_prompt = """You are a SQL Query Generator.

Your job:
- Generate SQL queries from natural language requirements
- Use standard SQL syntax
- Keep queries simple and readable
- Output ONLY the SQL query, no explanations
"""

# Agent 2: SQL Reviewer/Optimiser
reviewer_prompt = """You are a SQL Query Optimisation Expert.

Your job:
- Review SQL queries for performance issues
- Suggest specific improvements
- Identify missing indexes
- Recommend better approaches

Output format:
1. Issues found (if any)
2. Optimised query (if improvements needed)
3. Explanation of changes

If the query is already optimal, say "Query looks good!"
"""

print("✓ Two-agent system created!")

✓ Two-agent system created!


In [6]:
# The multi-agent workflow
def generate_and_review_sql(requirements):
    """
    Two-step process: Generate → Review
    """
    print("🔄 Step 1: Generating SQL...\n")

    generator = client.models.generate_content(
          model='gemini-2.5-flash-lite',
          contents='SQL Query Generator',
          config=types.GenerateContentConfig(
              system_instruction=requirements,
              tools=[{'code_execution': {}}]
          )
    )

    generated_sql = generator.text
    generated_sql = generated_sql.replace('```sql', '').replace('```', '').strip()

    print("📝 Generated SQL:")
    print("-"*80)
    print(generated_sql)
    print("-"*80)
    print("\n🔄 Step 2: Reviewing and optimising...\n")

    # Agent 2: Review

    reviewer = client.models.generate_content(
        model='gemini-2.5-flash-lite',
        contents='SQL Query Reviewer',
        config=types.GenerateContentConfig(
            system_instruction=f"Review this SQL query:\n\n{generated_sql}",
            tools=[{'code_execution': {}}]
    )
    )

    print("🔍 Review Results:")
    print("="*80)
    print(reviewer.text)
    print("="*80)

    return generated_sql, reviewer

# Test it
requirements = """
Get all orders from the last 30 days where the total is greater than £1000,
showing customer name, order date, and total amount.
Tables: orders (order_id, customer_id, order_date, total_amount)
        customers (customer_id, customer_name)
"""

sql, review = generate_and_review_sql(requirements)

🔄 Step 1: Generating SQL...

📝 Generated SQL:
--------------------------------------------------------------------------------
I can help you generate that SQL query!

To get all orders from the last 30 days where the total is greater than £1000, showing customer name, order date, and total amount, I need to join the `orders` and `customers` tables.

Here's the SQL query:


SELECT
    c.customer_name,
    o.order_date,
    o.total_amount
FROM
    orders AS o
JOIN
    customers AS c
ON
    o.customer_id = c.customer_id
WHERE
    o.order_date >= DATE('now', '-30 days') AND o.total_amount > 1000;


**Explanation:**

*   **`SELECT c.customer_name, o.order_date, o.total_amount`**: This specifies the columns you want to retrieve: the customer's name from the `customers` table (aliased as `c`), and the order date and total amount from the `orders` table (aliased as `o`).
*   **`FROM orders AS o JOIN customers AS c ON o.customer_id = c.customer_id`**: This joins the `orders` table with the `cu